# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Samvit Sutodiya
- Arya Nair
- Linwei Lu
- Jenny Zhang

# Group Members IDs

- A########
- A########
- A########
- A14854801

# Research Question

Does the amount of Medicare one receives for a specific illness differ from state to state based on the state’s population?

## Background and Prior Work

Medicare is a national health insurance program that covers health care costs for adults over the age of 65 and younger people with specific disabilities. Medicare has four parts that help cover certain services such as Hospital Insurance, Medical Insurance, prescription drug coverage and a Medicare Advantage which is an all in one plan. The main benefit of Medicare is that it provides health coverage regardless of the person’s income, thus benefiting people with low income by helping those who are unable to afford medical insurance (“Acute Inpatient PPS”, 2019).

But unfortunately this isn’t always the case. Studies such as ‘Medicare Utilization in the United States: PSRO and Regional Impacts’ by DL Rothberg, J Pinto and PM Gertman show how geographical factors affect medicare in the United States. Factors such as demographic and socio-economics play a role in determining the amount of medicare one receives. In this project we explore how population plays a role in determining the amount of medicare one receives. 

References (include links):
- 1) “Acute Inpatient PPS”, 2019 (https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AcuteInpatientPPS/index.html?redirect=/AcuteInpatientPPS/)
- 2) ‘Medicare Utilization in the United States: PSRO and Regional Impacts’ by DL Rothberg, J Pinto and PM Gertman (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1072167/)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

- Dataset Name: Inpatient Prospective Payment System (IPPS) Provider Summary for the Top 100 Diagnosis-Related Groups (DRG) - FY2011
- Link to the dataset: https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3
- Number of observations: 163065

The Inpatient Prospective Payment System is a payment system for inpatient care and stays covered by medical insurance (Medicare). The data table includes Total Discharges, Average Covered Discharges, Average Total Payments, and Average Medicare Payments which help extract the amount of Medicare provided for a specific illness. In the columns of the dataset, it involves DRG to classify the system grouping similar clinical conditions and procedures and the provider city. Hence, the said data tables help us to answer the question of how much the patients pay for the different illness in each state.

- Dataset Name: National, State, and Puerto Rico Commonwealth Totals Datasets: Population, population change, and estimated components of population change: April 1, 2010 to July 1, 2018
- Link to the dataset: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html
- Number of observations: 57

This dataset contains data from the 2010 US Census, as well as estimated yearly population, population change, and estimated components of population change of each state from 2010-2018.

- Dataset Name: CSV of States and State Abbreviations
- Link to the dataset: http://www.fonz.net/blog/archives/2008/04/06/csv-of-states-and-state-abbreviations/
- Number of observations: 51

This dataset contains all the US states and their abbreviations.

We plan to combine these databases by adding the populations for each provider state to analyze whether the population of a state affects the amount of Medicare one receives. Since the first dataset uses state abbreviations while the census uses their names, we will be using a third dataset to match abbreviations to state names.

# Setup

In [43]:
import pandas as pd

# Data Cleaning

Describe your data cleaning steps here.

First, we will read in the dataset.

In [44]:
df = pd.read_csv('Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_the_Top_100_Diagnosis-Related_Groups__DRG__-_FY2011.csv')

Next, we will drop the columns that we won't be using in our analysis, which are all the provider-related fields (aside from the state) and the hospital region. Some of the column names have extra spaces around them, so we will rename those columns.

In [45]:
df.drop(columns = ['Provider Id', 'Provider Name', 'Provider Street Address', 
                   'Provider City', 'Provider Zip Code', 'Hospital Referral Region Description'],
                 inplace = True)
columns = ['DRG Definition', 'State', 'Total Discharges', 'Average Covered Charges',
           'Average Total Payments', 'Average Medicare Payments']
df.columns = columns

Next, we need to combine all the data for each state. We will be analyzing different illnesses/conditions separately, so each state will have one row for each illness containing the Average Covered Charges (the provider's average charge for services) and Average Medicare Payments (the amount covered by Medicare, not including co-payment and deductibles). We will keep the Total Payments column as well, in case we want to analyze it separately later. Using the number of total discharges from each hospital, we will calculate each of these average amounts for each illness within each state.

Using these average values, we will create two new dataframes to perform analysis on. The first dataframe will contain the average amounts for each state and condition. The second dataframe will only contain the average values for each state, for the purpose of simplifying our overall analysis.

In [46]:
# construct the rows of the first dataframe, separated by condition and state
rows_condition = list()
for name, group in df.groupby(['State', 'DRG Definition']):
    total_discharges = group['Total Discharges'].sum()
    avg_charges = (group['Average Covered Charges'] * group['Total Discharges']).sum() / total_discharges
    avg_total = (group['Average Total Payments'] * group['Total Discharges']).sum() / total_discharges
    avg_medicare = (group['Average Medicare Payments'] * group['Total Discharges']).sum() / total_discharges
    rows_condition.append([name[1], name[0], total_discharges, avg_charges, avg_total, avg_medicare])

# construct the rows of the second dataframe, separated by state
rows_state = list()
for name, group in df.groupby(['State']):
    total_discharges = group['Total Discharges'].sum()
    avg_charges = (group['Average Covered Charges'] * group['Total Discharges']).sum() / total_discharges
    avg_total = (group['Average Total Payments'] * group['Total Discharges']).sum() / total_discharges
    avg_medicare = (group['Average Medicare Payments'] * group['Total Discharges']).sum() / total_discharges
    rows_state.append([name, total_discharges, avg_charges, avg_total, avg_medicare])
    
df_condition = pd.DataFrame(rows_condition, columns = columns)
df_state = pd.DataFrame(rows_state, columns = ['State', 'Total Discharges', 'Average Covered Charges',
                                     'Average Total Payments', 'Average Medicare Payments'])

Our df_condition dataframe currently contains data for the top 100 Diagnosis-Related Groups (DRG). To simplify our analysis, we will narrow down our data to the top 10 DRG, based on number of total discharges.

In [58]:
rows_DRG = list()
for name, group in df.groupby(['DRG Definition']):
    rows_DRG.append([name, group['Total Discharges'].sum()])
df_DRG = pd.DataFrame(rows_DRG, columns = ['DRG Definition', 'Total Discharges'])
top = df_DRG.nlargest(10, 'Total Discharges')['DRG Definition']
df_condition = df_condition[df_condition['DRG Definition'].isin(top)]

Next, we will read in the state population dataset. The population dataset uses the full names of states, but our Medicare dataset uses their abbreviations. To match populations with states, we will use a third dataset containing the abbreviations of states to add the abbreviations to the population dataset.

In [47]:
df_population = pd.read_csv('nst-est2018-alldata.csv')
df_states = pd.read_csv('states.csv')

# drop all columns except the two necessary columns
df_population = df_population[['NAME', 'POPESTIMATE2018']]

# rename the columns to merge with df_states
df_population.columns = ['State', 'Population']
df_population = df_population.merge(df_states, on='State')

# rename the columns again to merge with Medicare dataframes
df_population.drop(columns = ['State'], inplace = True)
df_population.columns = ['Population', 'State']

df_condition = df_condition.merge(df_population, on='State')
df_state = df_state.merge(df_population, on='State')

Finally, we will add a column for the ratio of the amount covered by medicare to the total charges from the provider.

In [49]:
df_state['Average % Covered'] = df_state['Average Medicare Payments'] / df_state['Average Covered Charges']
df_condition['Average % Covered'] = df_condition['Average Medicare Payments'] / df_condition['Average Covered Charges']

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*